In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import string
import math
import nltk
from nltk.corpus import words
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


import torch
import torch.nn as nn
import torchtext

OSError: [WinError 127] The specified procedure could not be found

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# !pip install pyenchant
# !apt-get install -y libenchant-2-2
import enchant
from enchant.tokenize import get_tokenizer
from enchant.tokenize import basic_tokenize

In [4]:
# !pip install tensorflow
from tensorflow.keras.utils import pad_sequences

In [5]:
dataset = './dataset/ielts-writing-essays.csv'

In [ ]:
# Load dataset & Prepare Dataframes
df = pd.read_csv(dataset)
display(df.head())
sentences = pd.DataFrame(df['Essay'])
scores = pd.DataFrame(df['Overall'])

# Define functions
def tokenize_and_filter_sentence(sentence):
    tokenizer = enchant.tokenize.get_tokenizer("en_GB")
    tokens = list(tokenizer(sentence))
    tokens = [token[0] for token in tokens]
    tokens = [token.lower() for token in tokens]
    return tokens

def convert_to_sequence(sentence, word_index):
    tokens = tokenize_and_filter_sentence(sentence)
    return [word_index.get(token, word_index['<OOV>']) for token in tokens]

def alternative_word2vec(sen, model):
    return [model.wv[word] for word in sen if word in model.wv]

def calculate_average_length(sentences):
    total_len = 0
    for sentence in sentences:
        total_len += len(sentence)
    average_len = int(round(total_len / len(sentences), 0))
    return average_len

def pad_word2vec(sentence_vectors, maxlen, vector_size):
    sentence_length = len(sentence_vectors)
    if sentence_length > maxlen:
        return np.array(sentence_vectors[:maxlen])
    else:
        padding = [np.zeros(vector_size) for _ in range(maxlen - sentence_length)]
        return np.array(sentence_vectors + padding)

,Essay,Overall
0,"Between 1995 and 2010, a study was conducted r...",5.5
1,Poverty represents a worldwide crisis. It is t...,6.5
2,The left chart shows the population change hap...,5.0
3,Human beings are facing many challenges nowada...,5.5
4,Information about the thousands of visits from...,7.0


KeyboardInterrupt: 

In [11]:
class LSTMModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim,hidden_dim, num_layers, dropout_rate, embedding_layer):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(embedding_layer, hidden_dim, num_layers=num_layers, 
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)          
        output = self.dropout(output) 
        prediction = self.fc(output)
        return prediction, hidden
    
    def init_weights(self):
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim, 
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell
    
    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# PARAMS
vocab_size = len(vocab)
embedding_dim = 1024             # 400 in the paper
hidden_dim = 256                # 1150 in the paper
num_layers = 3                   # 3 in the paper
dropout_rate = 0.2                            
lr = 1e-3
input_layer = 1

In [ ]:
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')